# TP5
## CEIA - Cohorte 7 - 2022
## Matias Porra

##### 1. Implementar el detector de fondo naive usando la mediana como estimador. El algoritmo debe recibir el parámetro N (cantidad de 7 frames utilizados para la estimación) y el intervalo de tiempo para recalcular el fondo.
##### 2. Se deben generar las mascaras de foreground y aplicarlas a los frames para segmentar los objetos en movimiento.

In [6]:
import cv2 as cv
import numpy as np
import time
import os
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (16.0, 16.0)

In [7]:
class Naive:
    
    def __init__(self, n_frames, time_interval, filename):
        self.capture = cv.VideoCapture(filename)        
        if not self.capture.isOpened:
            print('Falla al abrir el archivo: ' + filename)
            exit(0)
        
        self.n_frames = n_frames
        self.fps = self.capture.get(cv.CAP_PROP_FPS)
        self.interval = time_interval * self.fps 
        self.frame_width = int(self.capture.get(3))
        self.frame_height = int(self.capture.get(4))
        self.output_path = "./output/" 
        self.setBackground()
    
    def get_Background(self):
        frame_n = int(self.capture.get(cv.CAP_PROP_FRAME_COUNT)) * np.random.uniform(size=n_frames)
        frames = []
        for frame_num in frame_n:
            self.capture.set(cv.CAP_PROP_POS_FRAMES, frame_num)
            ret, frame = self.capture.read()
            frames.append(frame)

        background = np.median(frames, axis=0).astype(dtype=np.uint8)
        return background
    
    def setBackground(self, frame_queue=None):
        if frame_queue:
            frame_n = len(frame_queue) * np.random.uniform(size=self.n_frames)
            frame_n = frame_n.astype(dtype=np.uint8)
            bg_frames = np.array(frame_queue)[frame_n]
            self.background = np.median(bg_frames, axis=0).astype(dtype=np.uint8)
        else:
            self.background = self.get_Background()
        
    def getBackground(self):
        return self.background        
        
    def add(self):
        fourcc = cv.VideoWriter_fourcc(*'mp4v')
        output_filename = "output_nbs.mp4"
        output = cv.VideoWriter(self.output_path + "/" + output_filename,
                               fourcc,
                               self.fps,
                               (self.frame_width, self.frame_height),
                               False)
        # Reseteamos los frames
        self.capture.set(cv.CAP_PROP_POS_FRAMES, 0)
        start = time.time()
        gray_background = cv.cvtColor(self.background, cv.COLOR_BGR2GRAY)
        frame_queue = []
        ret, frame = self.capture.read()
        while ret:
            frame_queue.append(frame)
            # recalculo el fondo
            if len(frame_queue) % self.interval == 0:
                self.setBackground(frame_queue)
                gray_background = cv.cvtColor(self.background, cv.COLOR_BGR2GRAY)
                frame_queue = []
            frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            dframe = cv.absdiff(frame, gray_background)
            th, dframe = cv.threshold(dframe, 30, 255, cv.THRESH_BINARY)
            output.write(dframe)

            ret, frame = self.capture.read()
     

In [11]:
n_frames = 60
time_interval = 20 # in seconds
output_path = "./output/"
FILENAME = 'videos/vtest.avi' 

nbs = Naive(n_frames=n_frames, time_interval=time_interval, filename=FILENAME)
nbs.add()
capture = cv.VideoCapture(output_path + "output_nbs.mp4")
capture.set(cv.CAP_PROP_POS_FRAMES, 540)

True

##### 3. Comparar con alguno de los métodos vistos en la practica basados en mezcla de gaussianas

In [9]:
class MOG2:
    
    def __init__(self, filename):
        self.capture = cv.VideoCapture(filename)    
        if not self.capture.isOpened:
            print('Falla al abrir el archivo: ' + filename)
            exit(0)
            
        self.backSub = cv.createBackgroundSubtractorMOG2()
        
        self.fps = self.capture.get(cv.CAP_PROP_FPS)
        self.frame_width = int(self.capture.get(3))
        self.frame_height = int(self.capture.get(4))
        self.output_path = "./output/"
    
    def add(self, output_filename=None):
        fourcc = cv.VideoWriter_fourcc(*'mp4v')
        output = cv.VideoWriter(self.output_path + "/" + output_filename,
                               fourcc,
                               self.fps,
                               (self.frame_width, self.frame_height),
                               False)

        self.capture.set(cv.CAP_PROP_POS_FRAMES, 0)
        start = time.time()      
        ret, frame = self.capture.read()
        while ret:
            # Leemos un frame
            if frame is None:
                break
            fgMask = self.backSub.apply(frame)
            output.write(fgMask)
            ret, frame = self.capture.read()
        

In [10]:
mog2 = MOG2(FILENAME)
mog2.add(output_filename="mog2.mp4")
capture = cv.VideoCapture(output_path + "/mog2.mp4")
capture.set(cv.CAP_PROP_POS_FRAMES, 540)


True

## Conclusion
##### Naive detecta mejor el fondo que MOG2, quiza con una erosion para eliminar el ruido de alrededor de los objetos MOG2 mejoraria